<a href="https://colab.research.google.com/github/ldjlammers/Hide-and-Seek-CS4240/blob/master/Hide_and_Seek_Reproducibility_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reproducing Hide and Seek method for Classification and Localisation 

In this notebook we try to reproduce a method called "Hide and Seek", proposed by Singh and Lee in their 2017 paper '[Hide-and-Seek](https://arxiv.org/abs/1704.04232): Forcing a network to be Meticulous for weakly-supervised Object and Action Localization'. Blah blah blah 

---

# What is Hide and Seek?

Instead of making algoritmic changes, or relying on external data, the Hide and Seek method makes changes to the input image. The key idea is to hide patches of the training images rondomly, forcing the network to seek other relevant parts when the most discriminative part is hidden. 

In order to check wether the network has resorted to other areas of a photo then the most discriminative parts to 



---

## Running the reproducibility project in Google Colab

In order to run the experiment in Google Colab we had to mount the drive in google drive. 


In [0]:
# Mounting the Google drive 
import os
from google.colab import drive
drive.mount('/content/gdrive')

# Specifying the path to the  project folder directory
!ls '/content/gdrive/My Drive/Hide and Seek' 
root_path = 'gdrive/My Drive/Hide and Seek' 
path ='/content/gdrive/My Drive/Hide and Seek'
os.chdir(path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
 3.jpg			   'HaS temp.ipynb'
 3.png			    Hide-and-Seek_Reproducibility_Project.ipynb
 AlexNetScheme.png	    ImgswithBB.xlsx
 Annotation.tar.gz	    INDL
 data			    pretrained_models
 final_synids.xlsx	    Reproducibility_Setup.ipynb
'googlenet measurement 1'   save_trained



---

## Architectures: AlexNet and GoogLeNet

The AlexNet architechture was first proposed by *Krizhevsky et al.* in their [original paper](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf) called "ImageNet Classification with Deep Convolutional
Neural Networks". The network was created for image classification as an entry to the ImageNet LSVRC-2010 contest.  The Hide and Seek paper, which is reproduced in this notebook, does make a few changes to the original AlexNet architecture. The layers after conv5 (i.e., pool5 to prob) resulting in a mapping resolution of 13 × 13, are all removed. 

![AlexNet Schematic](https://drive.google.com/uc?export=view&id=1Rj-GGuBwexK39x3fKgBwOKYwUHXcgvdO)

Instead, after conv5, a convolutional layer of size 3 × 3, stride 1, pad 1 with 1024 units, followed by a GAP layer and a softmax layer, are added. 

The weights are initialized in each layer from a zero-mean Gaussian distribution with standard deviation 0.01. The neuron biases are initialized in the second, fourth, and fifth convolutional layers, as well as in the fully-connected hidden layers, with the constant 1.



In [0]:
import torch
import torch.nn as nn
import torch.nn.init as init
from collections import OrderedDict

class AlexNet(nn.Module):

    def __init__(self, num_classes=50, init_weights=True):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(OrderedDict([
            ('conv1', nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0)),
            ('Relu1', nn.ReLU(inplace=True)),
            ('maxpool1', nn.MaxPool2d(kernel_size=3, stride=2)),
            ('conv2', nn.Conv2d(96, 256, kernel_size=5, padding=2)),
            ('Relu2', nn.ReLU(inplace=True)),
            ('maxpool2', nn.MaxPool2d(kernel_size=3, stride=2)),
            ('conv3', nn.Conv2d(256, 384, kernel_size=3, padding=1)),
            ('Relu3', nn.ReLU(inplace=True)),
            ('conv4', nn.Conv2d(384, 384, kernel_size=3, padding=1)),  
            ('Relu4', nn.ReLU(inplace=True)),
            ('conv5', nn.Conv2d(384, 256, kernel_size=3, padding=1)), 
            ('Relu4', nn.ReLU(inplace=True)),
        ]))

        self.extraconvs = nn.Sequential(OrderedDict([
          ('extraconv1', nn.Conv2d(256, 512, kernel_size=3, padding=1)),                                           
          ('extraconv2', nn.Conv2d(256, 1024, kernel_size=3, padding=1))
        ]))
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))

        self.classifier = nn.Sequential(OrderedDict([
            ('extrafc', nn.Linear(1024 * 1 * 1, num_classes)),
            ('softmax', nn.Softmax(dim=1))                                       
        ]))

        if init_weights:
            self._initialize_weights()

    def _initialize_weights(self):
        bias_conv = ['conv2', 'conv4', 'conv5']
        for name, module in self.named_modules():
            if isinstance(module, nn.Conv2d):
                init.normal_(module.weight, std=1e-2)
                if name in bias_conv:
                    init.constant_(module.bias, 1)
            elif isinstance(module, nn.Linear):
                init.normal_(module.weight, std=1e-2)
                init.constant_(module.bias, 1)

    def forward(self, x):
        x = self.features(x)
        x = self.extraconvs(x)
        feat_maps = x
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x, feat_maps

For GoogLeNet, we removed the layers after inception4e (i.e., pool4 to prob), resulting in a mapping resolution of 14 × 14. 

![alt text](https://drive.google.com/uc?id=1arovt362S3Ud8mcVOP6ZM-Ws2suPZj2E)

In [0]:
# from __future__ import division
from collections import namedtuple
import warnings
import torch.nn.functional as F
from torch.jit.annotations import Optional, Tuple
from torch import Tensor

class Inception(nn.Module):

    def __init__(self, in_channels, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj,
                 conv_block=None):
        super(Inception, self).__init__()
        if conv_block is None:
            conv_block = BasicConv2d
        self.branch1 = conv_block(in_channels, ch1x1, kernel_size=1)

        self.branch2 = nn.Sequential(
            conv_block(in_channels, ch3x3red, kernel_size=1),
            conv_block(ch3x3red, ch3x3, kernel_size=3, padding=1)
        )

        self.branch3 = nn.Sequential(
            conv_block(in_channels, ch5x5red, kernel_size=1),
            # Here, kernel_size=3 instead of kernel_size=5 is a known bug.
            # Please see https://github.com/pytorch/vision/issues/906 for details.
            conv_block(ch5x5red, ch5x5, kernel_size=3, padding=1)
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1, ceil_mode=True),
            conv_block(in_channels, pool_proj, kernel_size=1)
        )
        

    def _forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)

        outputs = [branch1, branch2, branch3, branch4]
        return outputs

    def forward(self, x):
        outputs = self._forward(x)
        return torch.cat(outputs, 1)


class InceptionAux(nn.Module):

    def __init__(self, in_channels, num_classes, conv_block=None):
        super(InceptionAux, self).__init__()
        if conv_block is None:
            conv_block = BasicConv2d
        self.conv = conv_block(in_channels, 128, kernel_size=1)

        self.fc1 = nn.Linear(2048, 1024)
        self.fc2 = nn.Linear(1024, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):       
        x = F.adaptive_avg_pool2d(x, (4, 4))  # aux1: N x 512 x 14 x 14, aux2: N x 528 x 14 x 14       
        x = self.conv(x)                      # aux1: N x 512 x 4 x 4, aux2: N x 528 x 4 x 4      
        x = torch.flatten(x, 1)               # N x 128 x 4 x 4        
        x = F.relu(self.fc1(x), inplace=True) # N x 2048        
        x = F.dropout(x, 0.7, training=self.training)   # N x 1024        
        x = self.fc2(x)                       # N x 1024
        x = self.softmax(x)
        return x                              # N x 1000 (num_classes)


class BasicConv2d(nn.Module):

    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)


GoogLeNetOutputs = namedtuple('GoogLeNetOutputs', ['logits', 'aux_logits2', 'aux_logits1'])
GoogLeNetOutputs.__annotations__ = {'logits': Tensor, 'aux_logits2': Optional[Tensor],
                                    'aux_logits1': Optional[Tensor]}
_GoogLeNetOutputs = GoogLeNetOutputs

class GoogLeNet(nn.Module):
    __constants__ = ['aux_logits']

    def __init__(self, num_classes=50, aux_logits=True, init_weights=True, blocks=None):
        super(GoogLeNet, self).__init__()
        if blocks is None:
            blocks = [BasicConv2d, Inception, InceptionAux]
        assert len(blocks) == 3
        conv_block = blocks[0]
        inception_block = blocks[1]
        inception_aux_block = blocks[2]

        self.aux_logits = aux_logits

        self.features1 = nn.Sequential(OrderedDict([
          ('conv1', conv_block(3, 64, kernel_size=7, stride=2, padding=3)),
          ('maxpool1', nn.MaxPool2d(3, stride=2, ceil_mode=True)),
          ('conv2', conv_block(64, 64, kernel_size=1)),
          ('conv3', conv_block(64, 192, kernel_size=3, padding=1)),
          ('maxpool2', nn.MaxPool2d(3, stride=2, ceil_mode=True)),
        ]))

        self.inception3 = nn.Sequential(OrderedDict([
          ('incep3a', inception_block(192, 64, 96, 128, 16, 32, 32)),     
          ('incep3b', inception_block(256, 128, 128, 192, 32, 96, 64)),  
          ('maxpool3', nn.MaxPool2d(3, stride=2, ceil_mode=True)),        
        ]))

        self.inception4a = inception_block(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = inception_block(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = inception_block(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = inception_block(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = inception_block(528, 256, 160, 320, 32, 128, 128)

        self.extraconv = nn.Conv2d(832, 1024, kernel_size=3, padding=1)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))

        self.classifier = nn.Sequential(OrderedDict([
          ('extrafc', nn.Linear(1024 * 1 * 1, num_classes)),
          ('softmax', nn.Softmax(dim=1))                                       
        ]))

        if aux_logits:
            self.aux1 = inception_aux_block(512, num_classes)
            self.aux2 = inception_aux_block(528, num_classes)
        else:
            self.aux1 = None
            self.aux2 = None

        if init_weights:
            self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                import scipy.stats as stats
                X = stats.truncnorm(-2, 2, scale=0.01)
                values = torch.as_tensor(X.rvs(m.weight.numel()), dtype=m.weight.dtype)
                values = values.view(m.weight.size())
                with torch.no_grad():
                    m.weight.copy_(values)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _forward(self, x):        
        # type: (Tensor) -> Tuple[Tensor, Optional[Tensor], Optional[Tensor]]       
        x = self.features1(x)       # Out: N x 192 x 28 x 28
        x = self.inception3(x)      # Out: N x 480 x 14 x 14
        x = self.inception4a(x)     # Out: N x 512 x 14 x 14       
        aux1 = torch.jit.annotate(Optional[Tensor], None) 
        if self.aux1 is not None:
            if self.training:
                aux1 = self.aux1(x)

        x = self.inception4b(x)     # Out: N x 512 x 14 x 14
        x = self.inception4c(x)     # Out: N x 512 x 14 x 14        
        x = self.inception4d(x)     # Out: N x 528 x 14 x 14       
        aux2 = torch.jit.annotate(Optional[Tensor], None) 
        if self.aux2 is not None:
            if self.training:
                aux2 = self.aux2(x)

        x = self.inception4e(x)     # Out: N x 832 x 14 x 14
        x = self.extraconv(x)
        feat_maps = x
        x = self.avgool(x)
        x = torch.flatten(x,1)
        x = self.classifier(x)      
        return x, aux2, aux1, feat_maps

    @torch.jit.unused
    def eager_outputs(self, x, aux2, aux1):
        # type: (Tensor, Optional[Tensor], Optional[Tensor]) -> GoogLeNetOutputs
        if self.training and self.aux_logits:
            return _GoogLeNetOutputs(x, aux2, aux1)
        else:
            return x

    def forward(self, x):
        # type: (Tensor) -> GoogLeNetOutputs
        x, aux1, aux2, feat_maps = self._forward(x)
        aux_defined = self.training and self.aux_logits
        if torch.jit.is_scripting():
            if not aux_defined:
                warnings.warn("Scripted GoogleNet always returns GoogleNetOutputs Tuple")
            return GoogLeNetOutputs(x, aux2, aux1), feat_maps
        else:
            return self.eager_outputs(x, aux2, aux1), feat_maps

In [0]:
from torchvision import transforms
import numpy as np
import cv2
from PIL import Image

def returnCAM(feature_conv, weight_softmax, class_idx):
    # generate the class activation maps upsample to 224x224
    size_upsample = (224,224)
    bz, nc, h, w = feature_conv.shape
    output_cam = []
    for idx in class_idx:
        cam = torch.matmul(weight_softmax[idx], feature_conv.view((nc, h*w)))
        cam = cam.view(h, w)
        cam = cam - torch.min(cam)
        cam_img = cam / torch.max(cam)
        cam_img = 255*cam_img
        cam_img = cam_img.to(torch.uint8)
        output_cam.append(cv2.resize(cam_img.numpy(), size_upsample))
    return output_cam


#net123 = 'AlexNet'
#alexnet = globals()[net123]()

# alexnet = AlexNet()
# googlenet = GoogLeNet()

# load test image of mountain
# x = Image.open(os.path.join(path, '3.png'))
# x = transforms.ToTensor()(x)[:3,:,:].view(-1,3,224,224)

# Outputs AlexNet
# output, _ = alexnet(x)
# weight_fc_alex = alexnet.classifier[0].weight

# # Outputs GoogLeNet
#output, feat_maps_google = googlenet(x)           # output is namedtuple
# weight_fc_google = googlenet.classifier[0].weight

# idx = range(1000)
# # generate class activation mapping for the top1 prediction
# CAMs_alex = returnCAM(feat_maps_alex, weight_fc_alex, [idx[0]])
# CAMs_google = returnCAM(feat_maps_google, weight_fc_google, [idx[0]])
# #print(CAMs_alex.shape)
# #print(CAMs_google.shape)
      
# # render the CAM and output
# # print('output CAM.png for the top1 prediction: %s'%classes[idx[0]])
# img = cv2.imread('3.png')
# height, width, _ = img.shape
# heatmap_al = cv2.applyColorMap(cv2.resize(CAMs_alex[0], (width, height)), cv2.COLORMAP_JET)
# result_al = heatmap_al * 0.3 + img * 0.5
# cv2.imwrite('CAM alexnet.png', result_al)
# heatmap_go = cv2.applyColorMap(cv2.resize(CAMs_google[0], (width, height)), cv2.COLORMAP_JET)
# result_go = heatmap_go * 0.3 + img * 0.5
# cv2.imwrite('CAM google.png', result_go)

---

## Data Prep: Hide a Patch

In order to train the network, certain patches of the training images are hidden with probability p = 0.5 ...........


In [0]:
import random
import matplotlib.pyplot as plt


def hide_patch(img):
    # get width and height of the image
    s = img.shape
    wd = s[1]
    ht = s[2]

    # possible grid size, 0 means no hiding
    grid_sizes=[0,16,32,44,56]

    # hiding probability
    hide_prob = 0.5

    # randomly choose one grid size
    grid_size= grid_sizes[random.randint(0,len(grid_sizes)-1)]

    # hide the patches 
    if grid_size > 0:
        for x in range(0, wd, grid_size):
            for y in range(0, ht, grid_size):
                x_end = min(wd, x + grid_size)  
                y_end = min(ht, y + grid_size)
                if random.random() <=  hide_prob:
                      img[0, x:x_end, y:y_end] = 0.4639      # Hides patch to zero, but has to be mean of dataset!!!!!
                      img[1, x:x_end, y:y_end] = 0.4415
                      img[2, x:x_end, y:y_end] = 0.3947
    return img
              
# z = Image.open(os.path.join(path, '3.png'))
# z = transforms.ToTensor()(z)[:3,:,:].view(-1,3,224,224)
# batchsize = 1
# for i in range(batchsize):
#   img = hide_patch(z[i])

# plt.imshow(img.permute(1,2,0))
# plt.show()

---

## Model Hyperparameter setting

Blah blah blah


In [0]:
class ModelArgs:
   """
    Passing the hyperparameters to the model
   """
   def __init__(self, netType='alexnet', hideProb=0.5, validate=0, pretrained=0, patchsize=16, nClasses=50, 
                epochs=55, start_epoch=0, batch_size=128, lr=0.01, momentum=0.9, print_freq=30, 
                weight_decay=5e-4, save_dir='save_trained', save_every=10, inputsize=224):
        self.save_every = save_every  # Saves checkpoints at every specified number of epochs
        self.save_dir = save_dir      # The directory used to save the trained models
        self.validate = validate      # validate model on validation set
        self.pretrained = pretrained
        self.weight_decay = weight_decay
        self.momentum = momentum
        self.lr = lr                  # Learning rate
        self.print_freq = print_freq
        self.batch_size = batch_size
        self.start_epoch = start_epoch
        self.epochs = epochs
        self.nClasses = nClasses      # number of classes
        self.patchsize = patchsize    # size of patches
        self.hideProb = hideProb      # probability of hiding patch
        self.netType = netType        # type of neural net
        self.inputsize = inputsize    # alex = 227 and google = 224

alexargs = ModelArgs('alexnet', epochs=55, inputsize=227)
googleargs = ModelArgs('googlenet', epochs=40, batch_size=128)



---

## Import training and test images


In [0]:
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
from tqdm import tqdm

im_path= os.path.join(path, 'data')

def load_images(modelargs=alexargs):
    trans = transforms.Compose([transforms.Resize((modelargs.inputsize,modelargs.inputsize)), 
                                transforms.ToTensor()])
    trainset = datasets.ImageFolder(os.path.join(im_path, 'train'), 
                                    transform=trans)
    return torch.utils.data.DataLoader(trainset, batch_size=modelargs.batch_size, num_workers=4, pin_memory=True, shuffle=True)


def load_valims(modelargs=alexargs):
    trans = transforms.Compose([transforms.Resize((modelargs.inputsize,modelargs.inputsize)), 
                                transforms.ToTensor()])
    valset = datasets.ImageFolder(os.path.join(im_path, 'test'), 
                                  transform=trans)
    return torch.utils.data.DataLoader(valset, batch_size=modelargs.batch_size, shuffle=False, num_workers=4, pin_memory=True)

#images, labels = next(iter(train_loader))



---

## Training and Validation

"We train the network from scratch for 55 and 40 epochs for AlexNet-GAP and GoogLeNet=GAP, respectively, with a batch size of 128 and initial learning rate of 0.01. We gradually decrease the learning rate to 0.0001." 

**AlexNet:**
"We trained our models using stochastic gradient descent with a batch size of 128 examples, momentum of 0.9, and weight decay of 0.0005. "

**GoogLeNet:**
"Our training used asynchronous stochastic gradient descent with 0.9 momentum [17], fixed learning rate schedule (decreasing the learning rate by 4% every 8 epochs). Polyak averaging [13] was used to create the final model used at inference time."

In [0]:
from tqdm import tqdm

def train(train_loader, model, criterion, optimizer, args, epoch=1, HaS=False):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for idx, data in enumerate(train_loader):
        inputs, labels = data

        # Hide patches
        if HaS:
          for input in inputs:
            input = hide_patch(input)
            
        
        # measure data loading time
        data_time.update(time.time() - end)
        input_var = inputs.cuda()
        # target = labels.cuda()
        # target_var = target.cuda()
        target_var = labels.cuda()

        # compute output
        if args == alexargs:
          outputs, _ = model(input_var)
          loss = criterion(outputs, target_var)
        elif args == googleargs:
          outputs, _ = model(input_var)
          loss = criterion(outputs.logits, target_var) + 0.3*criterion(outputs.aux_logits2, target_var) + 0.3*criterion(outputs.aux_logits1, target_var)
          outputs = outputs.logits 

        # compute gradient and do SGD step
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        outputs = outputs.float()
        loss = loss.float()

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, target_var)[0]       # was target
        prec5 = accuracy(outputs.data, target_var)[1]       # was target
        losses.update(loss.item(), inputs.size(0))  
        top1.update(prec1.item(), inputs.size(0))
        top5.update(prec5.item(), inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if idx % args.print_freq == 0:
            print('Epoch: [{}][{}/{}]\t'
                  'Loss {:.4f} ({:.4f})\t'
                  'Prec@1 {:.3f} ({:.3f})\t'
                  'Prec@5 {:.3f} ({:.3f})\t batch time: {:.2f}'.format(
                      epoch, idx, len(train_loader), losses.val, losses.avg, top1.val, top1.avg, top5.val, top5.avg, batch_time.val))

In [0]:
def validate(val_loader, model, criterion):

    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for idx, data in enumerate(val_loader):
            inputs, labels = data

            target = labels.cuda()
            input_var = inputs.cuda()
            target_var = target.cuda()

            # compute output
            outputs, _ = model(input_var)           # ignore featmaps output
            loss = criterion(outputs, target_var)

            outputs = outputs.float()
            loss = loss.float()

            # measure accuracy and record loss
            prec1 = accuracy(outputs.data, target)[0]
            prec5 = accuracy(outputs.data, target)[1]
            losses.update(loss.item(), inputs.size(0))
            top1.update(prec1.item(), inputs.size(0))
            top5.update(prec5.item(), inputs.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()


    print('Test\t  Prec@1: {top1.avg:.3f}\t Prec@5: {top5.avg:.3f} (Err: {error:.3f} )\n'
            .format(top1=top1,top5=top5,error=100-top1.avg))

    return top1.avg

def save_checkpoint(state, filename='checkpoint.th'):
    """Save the training model"""
    torch.save(state, filename)

In [0]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def accuracy(output, target, topk=(1,5)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res



---

## Running the Experiment

Learning rate decay milestones = [18 29 43 52 1e8] by 0.5 and start 1e-2

In [0]:
import time

def main(args, model, HaS=True):
    global best_prec1
    
    # Check the save_dir exists or not
    if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)

    model.cuda()

    # define loss function (criterion) and pptimizer
    criterion = nn.CrossEntropyLoss().cuda()

    optimizer = torch.optim.SGD(model.parameters(), args.lr,
                                momentum=args.momentum,
                                weight_decay=args.weight_decay)

    lr_schedule = torch.optim.lr_scheduler.MultiStepLR(optimizer,
                        milestones=[18, 29, 43, 52, 1e8], last_epoch=args.start_epoch - 1)

    if args.validate:
        print('evalution mode')
        model.load_state_dict(torch.load(os.path.join(args.save_dir, '%s.th' %args.netType)))
        best_prec1 = validate(load_valims(args), model, criterion)
        return best_prec1

    if args.pretrained:
        print('evalution of pretrained model')
        args.save_dir='pretrained_models'
        pretrained_model= '%s.th' %args.netType
        model.load_state_dict(torch.load(os.path.join(args.save_dir, pretrained_model)))
        best_prec1 = validate(load_valims(args), model, criterion)
        return best_prec1

    for epoch in range(args.start_epoch, args.epochs):

        # train for one epoch
        print('Training {} model'.format(args.netType))
        print('current lr {:.5e}'.format(optimizer.param_groups[0]['lr']))
        train(load_images(args), model, criterion, optimizer, args, epoch, HaS)
        lr_schedule.step()

        # evaluate on validation set
        prec1 = validate(load_valims(args), model, criterion)

        # remember best prec@1 and save checkpoint
        is_best = prec1 > best_prec1          # Boolean 
        best_prec1 = max(prec1, best_prec1)

        if epoch > 0 and epoch % args.save_every == 0:
            save_checkpoint(model.state_dict(), filename=os.path.join(args.save_dir, '%s_checkp.th' %args.netType))
        if is_best:
            save_checkpoint(model.state_dict(), filename=os.path.join(args.save_dir, '%s.th' %args.netType))

    return best_prec1


In [0]:
best_prec1 = 0
best_prec1 = main(googleargs, GoogLeNet())
print('The lowest error from {} model after {} epochs is {error:.3f}'.format(alexargs.netType,alexargs.epochs,error=100-best_prec1))

Training googlenet model
current lr 1.00000e-02
Epoch: [0][0/370]	Loss 6.2592 (6.2592)	Prec@1 0.781 (0.781)	Prec@5 9.375 (9.375)	 batch time: 50.45
Epoch: [0][30/370]	Loss 6.2591 (6.2590)	Prec@1 2.344 (1.865)	Prec@5 8.594 (12.475)	 batch time: 2.38
Epoch: [0][60/370]	Loss 6.2588 (6.2587)	Prec@1 0.000 (2.024)	Prec@5 13.281 (13.525)	 batch time: 42.50
Epoch: [0][90/370]	Loss 6.2538 (6.2576)	Prec@1 2.344 (1.983)	Prec@5 20.312 (14.269)	 batch time: 1.02
Epoch: [0][120/370]	Loss 6.2530 (6.2558)	Prec@1 0.781 (1.937)	Prec@5 17.188 (14.598)	 batch time: 30.94
Epoch: [0][150/370]	Loss 6.2566 (6.2537)	Prec@1 1.562 (2.178)	Prec@5 14.062 (14.580)	 batch time: 1.03
Epoch: [0][180/370]	Loss 6.2408 (6.2507)	Prec@1 3.125 (2.512)	Prec@5 14.062 (14.801)	 batch time: 1.05
Epoch: [0][210/370]	Loss 6.2248 (6.2485)	Prec@1 5.469 (2.814)	Prec@5 18.750 (15.070)	 batch time: 1.04
Epoch: [0][240/370]	Loss 6.2368 (6.2469)	Prec@1 4.688 (3.063)	Prec@5 12.500 (15.447)	 batch time: 1.05
Epoch: [0][270/370]	Loss 6.225

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f1341d13208>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 961, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 941, in _shutdown_workers
    w.join()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process


Epoch: [15][30/370]	Loss 5.9078 (5.9473)	Prec@1 28.125 (28.125)	Prec@5 43.750 (41.658)	 batch time: 1.33
Epoch: [15][60/370]	Loss 5.9808 (5.9432)	Prec@1 25.781 (28.663)	Prec@5 39.062 (42.328)	 batch time: 1.23
Epoch: [15][90/370]	Loss 5.9238 (5.9410)	Prec@1 29.688 (29.121)	Prec@5 42.969 (42.874)	 batch time: 1.21
Epoch: [15][120/370]	Loss 5.9375 (5.9413)	Prec@1 28.906 (29.300)	Prec@5 43.750 (42.885)	 batch time: 1.22
Epoch: [15][150/370]	Loss 5.9106 (5.9408)	Prec@1 33.594 (29.294)	Prec@5 42.188 (43.010)	 batch time: 1.31
Epoch: [15][180/370]	Loss 5.8789 (5.9376)	Prec@1 35.938 (29.593)	Prec@5 48.438 (43.288)	 batch time: 1.17
Epoch: [15][210/370]	Loss 5.9797 (5.9338)	Prec@1 24.219 (29.884)	Prec@5 40.625 (43.561)	 batch time: 1.20
Epoch: [15][240/370]	Loss 5.9429 (5.9329)	Prec@1 30.469 (29.921)	Prec@5 46.875 (43.666)	 batch time: 1.24
Epoch: [15][270/370]	Loss 5.9351 (5.9335)	Prec@1 29.688 (29.909)	Prec@5 44.531 (43.623)	 batch time: 1.24
Epoch: [15][300/370]	Loss 5.9830 (5.9313)	Prec@1 

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f1341d13208>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 961, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 941, in _shutdown_workers
    w.join()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process


Epoch: [16][30/370]	Loss 5.8842 (5.9133)	Prec@1 36.719 (31.628)	Prec@5 53.125 (44.153)	 batch time: 1.13
Epoch: [16][60/370]	Loss 5.9243 (5.9132)	Prec@1 29.688 (31.711)	Prec@5 40.625 (44.365)	 batch time: 1.25


...

Test	  Prec@1: 29.543	 Prec@5: 42.514 (Err: 70.457 )

Training googlenet model
current lr 1.00000e-03


Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f1341d13208>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 961, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 941, in _shutdown_workers
    w.join()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f1341d13208>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 961, in __del__
    self._shutdown_workers()
  File "/usr/local/l

Epoch: [18][0/370]	Loss 5.8889 (5.8889)	Prec@1 32.812 (32.812)	Prec@5 46.094 (46.094)	 batch time: 4.98
Epoch: [18][30/370]	Loss 5.9201 (5.8964)	Prec@1 35.156 (33.392)	Prec@5 40.625 (44.153)	 batch time: 1.16
Epoch: [18][60/370]	Loss 5.8519 (5.8964)	Prec@1 41.406 (33.517)	Prec@5 50.781 (44.173)	 batch time: 1.14
Epoch: [18][90/370]	Loss 5.8305 (5.8927)	Prec@1 39.844 (33.980)	Prec@5 49.219 (44.428)	 batch time: 1.21
Epoch: [18][120/370]	Loss 5.9298 (5.8873)	Prec@1 31.250 (34.472)	Prec@5 40.625 (44.699)	 batch time: 1.14
Epoch: [18][150/370]	Loss 5.8669 (5.8819)	Prec@1 36.719 (34.908)	Prec@5 42.188 (44.961)	 batch time: 1.16
Epoch: [18][180/370]	Loss 5.7940 (5.8782)	Prec@1 40.625 (35.204)	Prec@5 53.906 (45.153)	 batch time: 1.18
Epoch: [18][210/370]	Loss 5.7575 (5.8766)	Prec@1 46.094 (35.323)	Prec@5 54.688 (45.157)	 batch time: 1.16
Epoch: [18][240/370]	Loss 5.8315 (5.8758)	Prec@1 39.062 (35.425)	Prec@5 43.750 (45.261)	 batch time: 1.14
Epoch: [18][270/370]	Loss 5.8379 (5.8738)	Prec@1 39

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f1341d13208>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 961, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 941, in _shutdown_workers
    w.join()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f1341d13208>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 961, in __del__
Exception ignored in: <bound method _MultiProcess

Test	  Prec@1: 38.286	 Prec@5: 46.476 (Err: 61.714 )

Training googlenet model
current lr 1.00000e-03
Epoch: [19][0/370]	Loss 5.8721 (5.8721)	Prec@1 37.500 (37.500)	Prec@5 48.438 (48.438)	 batch time: 4.54
Epoch: [19][30/370]	Loss 5.7876 (5.8499)	Prec@1 40.625 (38.029)	Prec@5 50.000 (46.648)	 batch time: 1.25
Epoch: [19][60/370]	Loss 5.8920 (5.8559)	Prec@1 33.594 (37.154)	Prec@5 41.406 (46.094)	 batch time: 1.16
Epoch: [19][90/370]	Loss 5.9130 (5.8547)	Prec@1 35.938 (37.337)	Prec@5 42.969 (46.120)	 batch time: 1.18
Epoch: [19][120/370]	Loss 5.9068 (5.8526)	Prec@1 34.375 (37.468)	Prec@5 46.094 (46.365)	 batch time: 1.14
Epoch: [19][150/370]	Loss 5.9020 (5.8566)	Prec@1 33.594 (37.200)	Prec@5 42.188 (46.083)	 batch time: 1.15
Epoch: [19][180/370]	Loss 5.8305 (5.8611)	Prec@1 38.281 (36.878)	Prec@5 48.438 (45.822)	 batch time: 1.16
Epoch: [19][210/370]	Loss 5.8104 (5.8609)	Prec@1 42.188 (36.856)	Prec@5 49.219 (45.831)	 batch time: 1.31
Epoch: [19][240/370]	Loss 5.8779 (5.8608)	Prec@1 32.812

In [0]:
# path1 = '/content/gdrive/My Drive/Hide and Seek/data/test'
# x = os.listdir(path1)
# for element in x:
#   print(element)
#   images = os.listdir(os.path.join(path, element))
#   for i in range(len(images)):
#     if i < 75:
#       pass
#     else:
#       os.remove(os.path.join(os.path.join(path, element), images[i]))

# for element in x:
#   print(element)
#   images = os.listdir(os.path.join(path1, element))
#   print(len(images))
# hfhfhuhjg